In [1]:
import cv2 as cv
import numpy as np

In [2]:
def blend_images(image1, image2):
    print(image1.shape)
    print(image2.shape)
    image1_image2 = np.hstack((image1, image2))

    # generate Gaussian pyramid for image1
    image1_copy = image1.copy()
    gp_image1 = [image1_copy]
    for i in range(6):
        image1_copy = cv.pyrDown(image1_copy)
        gp_image1.append(image1_copy)


    # generate Gaussian pyramid for image2
    image2_copy = image2.copy()
    gp_image2 = [image2_copy]
    for i in range(6):
        image2_copy = cv.pyrDown(image2_copy)
        gp_image2.append(image2_copy)

    # generate Laplacian Pyramid for image1
    image1_copy = gp_image1[5]
    lp_image1 = [image1_copy]
    for i in range(5, 0, -1):
        gaussian_expanded = cv.pyrUp(gp_image1[i])
        laplacian = cv.subtract(gp_image1[i-1], gaussian_expanded)
        lp_image1.append(laplacian)

    # generate Laplacian Pyramid for image2
    image2_copy = gp_image2[5]
    lp_image2 = [image2_copy]
    for i in range(5, 0, -1):
        gaussian_expanded = cv.pyrUp(gp_image2[i])
        laplacian = cv.subtract(gp_image2[i-1], gaussian_expanded)
        lp_image2.append(laplacian)

    # Now add left and right halves of images in each level
    image1_image2_pyramid = []
    n = 0
    for image1_lap, image2_lap in zip(lp_image1, lp_image2):
        n += 1
        cols, rows, ch = image1_lap.shape
        laplacian = np.hstack((image1_lap[:, 0:int(cols/2)], image2_lap[:, int(cols/2):]))
        image1_image2_pyramid.append(laplacian)
    # now reconstruct
    image1_image2_reconstruct = image1_image2_pyramid[0]
    for i in range(1, 6):
        image1_image2_reconstruct = cv.pyrUp(image1_image2_reconstruct)
        image1_image2_reconstruct = cv.add(image1_image2_pyramid[i], image1_image2_reconstruct)

    return image1_image2_reconstruct